# Task 1: conformational relationships within a single ensemble

Relationships within an ensemble will be identified considering the structural features of single
conformations

### 0. Set-up

### 1. Load PDB structures of one single PED ensemble

### 2. Feature extraction (single conformation)
In the following we summarize the single conformation features that we are going to compute

| Feature       | Dimension     |
| :------------- |:-------------|
| Radius of gyration of the structure | Scalar |
| Relative accessible surface area (ASA) for each residue | Vector of size $N$      |
| Secondary structure (SS) for each residue and class | Vector of size $N$       |
|Residue distance matrix considering Cα atoms | Matrix of shape $N \times N$ (symmetric) |

where $N$ is the number of residues in one conformation.

### 3. Graph extraction
Nodes are a subset of representative conformations and edges represent their similarity (or distance). The similarity is calculated combining all feature values. Representative conformations are found by unsupervised
clustering and the number of clusters is identified automatically.

### 4. Pymol image
An Pymol image including the PDB structures corresponding to the graph nodes.
Centered (translate) the selected structures on the position(s) with the lowest feature
variance. Residues in each structure will be displayed (color, size, ...) based on their
feature variability within the ensemble.